In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns

from data_functions import *
from metric_functions import *
from helper_functions import *
from models import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Hiperparametry

In [2]:
dataset = load_data("./train")
labels = dataset.classes
dataset = torch.load("final_data.tar")
train, val = trai_val_split(dataset, 0.8)

In [3]:
n_convs_values = [2, 5, 10]
kernel_size_values = [3, 5, 7]
stride_values = [1, 2, 3]
padding_values = [1, 2, 3]

In [6]:
res = pd.DataFrame(columns=["n_convs", "kernel_size", "stride", "padding", "train_loss", "train_acc", "val_acc", "overfit"])
nets = {}

In [7]:
from tqdm import notebook as tqdm_notebook

for n_convs in tqdm_notebook.tqdm(n_convs_values):
    for kernel_size in tqdm_notebook.tqdm(kernel_size_values):
        for stride in tqdm_notebook.tqdm(stride_values):
            for padding in tqdm_notebook.tqdm(padding_values):
                net = CustomNet(num_classes=len(labels), input_size=48, n_convs=n_convs, kernel_size=kernel_size, stride=stride, padding=padding).to(device)
                nets[(n_convs, kernel_size, stride, padding)] = net
                train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True, num_workers=8)
                val_loader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True, num_workers=8)
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(net.parameters(), lr=0.001)
                loss_hist, train_eval_hist, val_eval_hist = train_model(net, train_loader, val_loader, criterion, optimizer, get_accuracy, device, 25, verbose=False)
                model_res = pd.DataFrame([[n_convs, kernel_size, stride, padding, loss_hist, train_eval_hist[-1], val_eval_hist[-1], train_eval_hist[-1]-val_eval_hist[-1]]], columns=["n_convs", "kernel_size", "stride", "padding", "train_loss", "train_acc", "val_acc", "overfit"])
                res = pd.concat([res, model_res])

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
display(res.sort_values(by="val_acc", ascending=False).head(10))